# Scikit-learn Beginner's Template

---

## Selected Machine Learning References
![TODO extend](https://img.shields.io/badge/TODO-extend-orange.svg)

### Scikit-learn

* Documentation: http://scikit-learn.org/stable/documentation.html
* User guide: http://scikit-learn.org/stable/user_guide.html

### ML and Python

* Python Data Science Handbook, Jake VanderPlas: https://github.com/jakevdp/PythonDataScienceHandbook

### ML in general

* List of HEP-ML resources: https://github.com/iml-wg/HEP-ML-Resources
* Stanford lecture series on Machine Learning by Andrej Karpathy: https://www.youtube.com/playlist?list=PLkt2uSq6rBVctENoVBg1TpCC7OQi31AlC

---

## Example Analysis

![TODO explain](https://img.shields.io/badge/TODO-explain-orange.svg)

### Preparatory Steps

In [ ]:
import math
import itertools

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set()
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

### Data Import

Use the Higgs data set from https://archive.ics.uci.edu/ml/datasets/HIGGS

In [ ]:
attributes = [
    'class_label', # 1 for signal, 0 for background
    'lepton1_pT',
    'lepton1_eta',
    'lepton1_phi',
    'missing_energy_mag',
    'missing_energy_phi',
    'jet1_pT',
    'jet1_eta',
    'jet1_phi',
    'jet1_btag',
    'jet2_pT',
    'jet2_eta',
    'jet2_phi',
    'jet2_btag',
    'jet3_pT',
    'jet3_eta',
    'jet3_phi',
    'jet3_btag',
    'jet4_pT',
    'jet4_eta',
    'jet4_phi',
    'jet4_btag',
    'm_jj',
    'm_jjj',
    'm_lv',
    'm_jlv',
    'm_bb',
    'm_wbb',
    'm_wwbb'
]

data = pd.read_csv('./data/higgs/HIGGS.csv',
                   header=None,
                   sep=',',
                   names=attributes,
                   usecols=range(0,22),
                   nrows=500000)

X = data.drop(['class_label'], axis=1)
y = (data['class_label']).astype(int)

In [ ]:
print('Dimensions of feature matrix X: ', X.shape)
print('Dimensions of target vector y:  ', y.shape)

print('\nTotal number of events in data sample: %d' % X.shape[0])
print('Number of signal events in data sample: %d (%.2f percent)' % (y[y==1].shape[0], y[y==1].shape[0]*100/y.shape[0]))
print('Number of backgr events in data sample: %d (%.2f percent)' % (y[y==0].shape[0], y[y==0].shape[0]*100/y.shape[0]))

![TODO idea](https://img.shields.io/badge/TODO-idea-yellow.svg) _compute correlation matrix (?) ..._

### Data Preprocessing

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

print('Mean values of transformed data: \n', scaler.mean_)
print('Variance of transformed data: \n', scaler.scale_)

![TODO idea](https://img.shields.io/badge/TODO-idea-yellow.svg) _PCA (?)_

### Data Split Into Training, Validation and Test Sample

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y,
#                                                    test_size=0.2,
#                                                    random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X, y, 
                                                  test_size=0.5,
                                                  random_state=42)

print('Number of training samples:   %d' % X_train.shape[0])
print('Number of validation samples: %d' % X_val.shape[0])

![TODO idea](https://img.shields.io/badge/TODO-idea-yellow.svg) _prepare data for proper cross-validation_

### Model Definition

In [ ]:
#default parameter settings

clf = RandomForestClassifier(n_estimators=750,
                             criterion='gini',
                             max_depth=None,
                             min_samples_split=500,
                             min_samples_leaf=1,
                             min_weight_fraction_leaf=0.0,
                             max_features='auto',
                             max_leaf_nodes=None,
                             min_impurity_split=1e-07,
                             bootstrap=True,
                             oob_score=False,
                             n_jobs=-1,
                             random_state=None,
                             verbose=1,
                             warm_start=False,
                             class_weight=None)

### Model Training

In [ ]:
clf = clf.fit(X_train, y_train)

print("Accuracy on training set:   {:.3f}".format(clf.score(X_train, y_train)))
print("Accuracy on validation set: {:.3f}".format(clf.score(X_val, y_val)))

In [ ]:
# feature importances

importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
    
# Print the feature ranking
print("Feature ranking:")
    
indices_featureNames = np.empty([X_train.shape[1]], dtype=object)
    
for f in range(X_train.shape[1]):
    indices_featureNames[f] = data.columns[indices[f]]
    print("\t%d. %s \t(%f)" % (f + 1,
                               indices_featureNames[f],
                               importances[indices[f]]))
    
plt.figure()
plt.title("Feature importances")
plt.bar(range(X_train.shape[1]),
        importances[indices],
        color="r",
        yerr=std[indices],
        align="center")
plt.xticks(range(X_train.shape[1]), indices_featureNames, rotation=90)
plt.xlim([-1, X_train.shape[1]])
plt.tight_layout()

---

### Model Evaluation on the Validation Sample

In [ ]:
y_val_score = clf.predict_proba(X_val)

#### MVA output distribution

In [ ]:
y_score_val_negClass, y_score_val_posClass = np.split(y_val_score,
                                                      2,
                                                      axis=1)

y_score_val_posClass_truePos = y_score_val_posClass[np.array(y_val==1)]
y_score_val_posClass_trueNeg = y_score_val_posClass[np.array(y_val==0)]

nbins = 100
plt.figure()

n_total, bins_total, patches_total = \
    plt.hist(y_val_score[:,1],
             bins=nbins,
             alpha=.25,
             color='black',
             label='MVA output')
    
n_trueNeg, bins_trueNeg, patches_trueNeg = \
    plt.hist(y_score_val_posClass_trueNeg,
             bins=nbins,
             alpha=0.5,
             color='#dd0000',
             label='true negative')
    
n_truePos, bins_truePos, patches_truePos = \
    plt.hist(y_score_val_posClass_truePos,
             bins=nbins,
             alpha=0.5,
             color='green',
             label='true positive')
    
plt.title('MVA output distribution (positive class)')
plt.xlim(-0.05, 1.05)
plt.xlabel('MVA output')
plt.ylabel('Entries')
plt.legend()

#### Cut efficiencies plot / MVA cut optimization

In [ ]:
MVAcut = np.empty((0))

plt.figure()
fig, ax1 = plt.subplots()
signal_efficiency = np.empty((0))
backgr_efficiency = np.empty((0))
for i in range(nbins):
    signal_efficiency = np.append(signal_efficiency, \
                                  np.sum(n_truePos[i:n_truePos.shape[0]]) / np.sum(n_truePos))
    backgr_efficiency = np.append(backgr_efficiency, \
                                  np.sum(n_trueNeg[i:n_trueNeg.shape[0]]) / np.sum(n_trueNeg))
    MVAcut = np.append(MVAcut, i/(nbins*1.0))
l1 = ax1.plot(MVAcut, signal_efficiency, label='signal efficiency', color='blue')
l2 = ax1.plot(MVAcut, backgr_efficiency, label='background efficiency', color='red')
ax1.set_xlabel('MVA cut')
ax1.set_ylabel('Efficiency')

ax2 = ax1.twinx()
significance_per_MVAcut = np.empty((0))
for i in range(nbins):
    significance_per_MVAcut = np.append(significance_per_MVAcut, \
                                        np.sum(n_truePos[i:n_truePos.shape[0]]) / \
                                        math.sqrt(np.sum(n_truePos[i:n_truePos.shape[0]] + \
                                                         n_trueNeg[i:n_trueNeg.shape[0]])))
    
l3 = ax2.plot(MVAcut, significance_per_MVAcut,
              label='significance',
              color='green')
pos_max = np.argmax(significance_per_MVAcut)
threshold_pos_max = pos_max/(nbins*1.0)
l4 = ax2.plot(pos_max/(nbins*1.0), significance_per_MVAcut[pos_max],
              label='max. significance for cut at %.2f' % threshold_pos_max,
              marker='o', markersize=10, fillstyle='none', mew=2, linestyle='none',
              color='#005500')
ax2.set_ylabel('Significance', color='green')
ax2.tick_params('y', colors='green')

plt.title('MVA cut efficiencies')
lall = l1+l2+l3+l4
labels = [l.get_label() for l in lall]
ax2.legend(lall, labels, loc='lower left')
plt.tight_layout()

#### ROC curve

In [ ]:
fpr, tpr, thresholds = roc_curve(y_val, y_val_score[:,1], pos_label=1)
roc_auc = roc_auc_score(y_val, y_val_score[:,1])

plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic curve')
    
# find and plot threshold closest to threshold_pos_max (i.e., the chosen
# working point)
close_threshold_pos_max = np.argmin(np.abs(thresholds-threshold_pos_max))

plt.plot(fpr[close_threshold_pos_max], tpr[close_threshold_pos_max], 'o', markersize=10,
        label="threshold at %.2f" % threshold_pos_max, fillstyle="none",
        mew=2)

plt.legend(loc=4)

#### Precision-recall curve

In [ ]:
precision = dict()
recall = dict()
average_precision = dict()

precision, recall, thresholds_PRC = \
    precision_recall_curve(y_val,
                           y_val_score[:,1])
    
average_precision = average_precision_score(y_val, y_val_score[:,1])
    
# Plot Precision-Recall curve
n_classes=1

plt.figure()
plt.plot(recall, precision, lw=2,
         label='Precision-recall curve of signal class (area = {1:0.2f})'
                ''.format(1, average_precision))

# find threshold closest to threshold_pos_max (i.e., the chosen
# working point)
close_optimum = np.argmin(np.abs(thresholds_PRC-threshold_pos_max))

plt.plot(recall[close_optimum], precision[close_optimum],
         'o',
         markersize=10,
         label="threshold at %.2f" % threshold_pos_max,
         fillstyle="none",
         mew=2)

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel(r'Recall $R=T_p / (T_p+F_n)$')
plt.ylabel(r'Precision $P=T_p / (T_p+F_p)$')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower right")

#### Confusion matrix

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    
    print('Generating confusion matrix...')
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes, rotation=45)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
# Compute confusion matrix
y_val_score_labels = (y_val_score[:,1] > threshold_pos_max)
cnf_matrix = confusion_matrix(y_val, y_val_score_labels)
np.set_printoptions(precision=2)

In [ ]:
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix,
                      classes=['negative class','positive class'],
                      title='Confusion matrix (non-normalized)')

In [ ]:
# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix,
                      classes=['negative class','positive class'],
                      normalize=True,
                      title='Normalized confusion matrix')

#### Classification report

In [ ]:
print(classification_report(y_val, y_val_score_labels,
                            target_names=['negative class','positive class']))

---

### Model Application to the Test Sample

#### MVA output distribution

#### ROC curve

#### Precision-recall curve

#### Confusion matrix

#### Classification report